## Import packages

In [1]:
import argparse
import json
import logging
import math
import os
import random
from pathlib import Path

import datasets
import nltk
import numpy as np
import torch
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from accelerate.utils import set_seed
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    SchedulerType,
    get_scheduler,
)

from tw_rouge import get_rouge
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/bosh/anaconda3/envs/adl/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/home/bosh/anaconda3/envs/adl/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/home/bosh/anaconda3/envs/adl/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
2022-04-30 18:21:31.010363: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit:

## Args

In [2]:
parser = argparse.ArgumentParser(
    description="Finetune a transformers model on a summarization task")
parser.add_argument(
    "--do_train",
    action="store_true",
    help="If passed, go through the trian process",
)
parser.add_argument(
    "--do_eval",
    action="store_true",
    help="If passed, go through the test process",
)
parser.add_argument(
    "--train_file", type=str, default="./data/train.jsonl", help="A csv or a json file containing the training data."
)
parser.add_argument(
    "--validation_file", type=str, default="./data/public.jsonl", help="A csv or a json file containing the validation data."
)
parser.add_argument(
    "--ignore_pad_token_for_loss",
    type=bool,
    default=True,
    help="Whether to ignore the tokens corresponding to " "padded labels in the loss computation or not.",
)
parser.add_argument(
    "--max_source_length",
    type=int,
    default=256,
    help="The maximum total input sequence length after "
    "tokenization.Sequences longer than this will be truncated, sequences shorter will be padded.",
)
# for t5 series model "summarize: "
parser.add_argument(
    "--source_prefix",
    type=str,
    default=None,
    help="A prefix to add before every source text " "(useful for T5 models).",
)
# 可不設
# Max number of processes when generating cache. Already cached shards are loaded sequentially
parser.add_argument(
    "--preprocessing_num_workers",
    type=int,
    default=None,
    help="The number of processes to use for the preprocessing.",
)
# not use the cached files and force the preprocessing to be applied every times
parser.add_argument(
    "--overwrite_cache", type=bool, default=True, help="Overwrite the cached training and evaluation sets"
)
parser.add_argument(
    "--max_target_length",
    type=int,
    default=64,
    help="The maximum total sequence length for target text after "
    "tokenization. Sequences longer than this will be truncated, sequences shorter will be padded."
    "during ``evaluate`` and ``predict``.",
)

# tutorial 用 beam search
parser.add_argument(
    "--num_beams",
    type=int,
    default=5,
    help="Number of beams to use for evaluation. This argument will be "
    "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.",
)
# 要加
parser.add_argument(
    "--pad_to_max_length",
    action="store_true",
    help="If passed, pad all samples to `max_length`. Otherwise, dynamic padding is used.",
)
parser.add_argument(
    "--model_name_or_path",
    type=str,
    default="google/mt5-small",
    help="Path to pretrained model or model identifier from huggingface.co/models.",
    # required=True,
)
# 不加
parser.add_argument(
    "--use_slow_tokenizer",
    action="store_true",
    help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).",
)
parser.add_argument(
    "--per_device_train_batch_size",
    type=int,
    default=4,
    help="Batch size (per device) for the training dataloader.",
)
parser.add_argument(
    "--per_device_eval_batch_size",
    type=int,
    default=4,
    help="Batch size (per device) for the evaluation dataloader.",
)
parser.add_argument(
    "--learning_rate",
    type=float,
    default=5e-5,
    help="Initial learning rate (after the potential warmup period) to use.",
)
parser.add_argument("--weight_decay", type=float,
                    default=0.0, help="Weight decay to use.")
parser.add_argument("--num_train_epochs", type=int, default=1,
                    help="Total number of training epochs to perform.")
parser.add_argument(
    "--max_train_steps",
    type=int,
    default=None,
    help="Total number of training steps to perform. If provided, overrides num_train_epochs.",
)
parser.add_argument(
    "--gradient_accumulation_steps",
    type=int,
    default=4,
    help="Number of updates steps to accumulate before performing a backward/update pass.",
)
parser.add_argument(
    "--lr_scheduler_type",
    type=SchedulerType,
    default="linear",
    help="The scheduler type to use.",
    choices=["linear", "cosine", "cosine_with_restarts",
             "polynomial", "constant", "constant_with_warmup"],
)
parser.add_argument(
    "--num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler."
)
parser.add_argument("--output_dir", type=str, default="./mt5_small_model",
                    help="Where to store the final model.")
parser.add_argument("--seed", type=int, default=31,
                    help="A seed for reproducible training.")
parser.add_argument(
    "--debug",
    action="store_true",
    help="Activate debug mode and run training only with a subset of data.",
)


args = parser.parse_args(
    # args=['--pad_to_max_length', '--do_train', '--debug']
    args=['--pad_to_max_length', '--do_train',]
)
print(args)




Namespace(config_name=None, debug=False, do_eval=False, do_train=True, gradient_accumulation_steps=4, ignore_pad_token_for_loss=True, learning_rate=5e-05, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, max_length=128, max_source_length=256, max_target_length=64, max_train_steps=None, model_name_or_path='google/mt5-small', num_beams=5, num_train_epochs=1, num_warmup_steps=0, output_dir='./mt5_small_model', overwrite_cache=True, pad_to_max_length=True, per_device_eval_batch_size=4, per_device_train_batch_size=4, preprocessing_num_workers=None, seed=31, source_prefix=None, summary_column=None, text_column=None, tokenizer_name=None, train_file='./data/train.jsonl', use_slow_tokenizer=False, val_max_target_length=64, validation_file='./data/public.jsonl', weight_decay=0.0)


## Initialize the Accelerator and logger

In [3]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Make one log on every process with the configuration for debugging.
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)

# Setup logging, we only want one process per machine to log things on the screen.
# accelerator.is_local_main_process is only True for one process per machine.
logger.setLevel(
    logging.INFO if accelerator.is_local_main_process else logging.ERROR)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()


04/30/2022 18:21:32 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: no



## Set seed and check output_dir

In [4]:
# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

# Handle the repository creation
if accelerator.is_main_process:
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

## Load Dataset

In [5]:
data_files = {}
if args.train_file is not None:
    data_files["train"] = args.train_file
if args.validation_file is not None:
    data_files["validation"] = args.validation_file
raw_datasets = load_dataset("json", data_files=data_files)

 # Trim a number of training examples 取 100 個 data 來用，正式訓練或預測時關掉
if args.debug:
    for split in raw_datasets.keys():
        raw_datasets[split] = raw_datasets[split].select(range(100))

print(raw_datasets)

04/30/2022 18:21:34 - WARNING - datasets.builder - Using custom data configuration default-d107d6620dd251dd
04/30/2022 18:21:34 - WARNING - datasets.builder - Reusing dataset json (/home/bosh/.cache/huggingface/datasets/json/default-d107d6620dd251dd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
100%|██████████| 2/2 [00:00<00:00, 151.60it/s]

DatasetDict({
    train: Dataset({
        features: ['date_publish', 'title', 'source_domain', 'maintext', 'split', 'id'],
        num_rows: 21710
    })
    validation: Dataset({
        features: ['date_publish', 'title', 'source_domain', 'maintext', 'split', 'id'],
        num_rows: 5494
    })
})


## Load pretrained model and tokenizer

In [6]:
# config & tokenizer
if args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "Please make sure model_name_or_path is not None"
    )
    

# mt5-small model
if args.model_name_or_path:
    model = AutoModelForSeq2SeqLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    logger.info("Training new model from scratch")
    model = AutoModelForSeq2SeqLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))
if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

prefix = args.source_prefix if args.source_prefix is not None else ""

loading configuration file https://huggingface.co/google/mt5-small/resolve/main/config.json from cache at /home/bosh/.cache/huggingface/transformers/97693496c1a0cae463bd18428187f9e9924d2dfbadaa46e4d468634a0fc95a41.dadce13f8f85f4825168354a04675d4b177749f8f11b167e87676777695d4fe4
Model config MT5Config {
  "_name_or_path": "google/mt5-small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.17.0",
  "use_cache": true,
  "vocab_size": 250112
}

loading configuration file https://huggingface.c

## Preprocessing the datasets.

In [7]:
column_names = raw_datasets["train"].column_names
dataset_columns = {"text_column": "maintext", "summary_column": "title"}
label_pad_token_id = -100 if args.ignore_pad_token_for_loss else tokenizer.pad_token_id

def preprocess_function(examples):
    if args.do_eval:
        inputs = examples[dataset_columns["text_column"]]
    elif args.do_train:
        inputs = examples[dataset_columns["text_column"]]
        targets = examples[dataset_columns["summary_column"]]
        
    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(
        inputs, max_length=args.max_source_length, padding="max_length", truncation=True)
    
    if args.do_train:
        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=args.max_target_length,
                            padding="max_length", truncation=True)

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore padding in the loss.
        if args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else label_pad_token_id) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def calculate_rouge(preds, refs, avg=True):
    preds = [pred.strip() +'\n' for pred in preds]
    refs = [ref.strip() +'\n' for ref in refs]
    return get_rouge(preds, refs, avg)


In [8]:
# First we tokenize all the texts

with accelerator.main_process_first():
    processed_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

print(train_dataset)
print(eval_dataset)


Running tokenizer on dataset: 100%|██████████| 6/6 [00:02<00:00,  2.44ba/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 21710
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5494
})


## Create DataLoaders

In [9]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
)

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, shuffle=False, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
)


## Optimizer


In [10]:
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(
    len(train_dataloader) / args.gradient_accumulation_steps) 

# args.max_train_steps = epoch * (num of data)/(batch size) / args.gradient_accumulation_steps
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(
        args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)


/home/bosh/anaconda3/envs/adl/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Prepare everything with our accelerator.

In [11]:
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

## Training

In [12]:
# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps
logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {args.num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")

progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0
best_eval_rouge = 0
eval_rouge_history = {"rouge-1":[], "rouge-2":[], "rouge-l":[]}

for epoch in range(starting_epoch, args.num_train_epochs):
    # Train
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)

        # update parameters (weight and bias) 
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            completed_steps += 1
    
    # Evaluation
    model.eval()
    # greedy decoding by calling greedy_search() if num_beams=1 and do_sample=False.
    # multinomial sampling by calling sample() if num_beams=1 and do_sample=True.
    # beam-search decoding by calling beam_search() if num_beams>1 and do_sample=False.
    # beam-search multinomial sampling by calling beam_sample() if num_beams>1 and do_sample=True.
    # diverse beam-search decoding by calling group_beam_search(), if num_beams>1 and num_beam_groups>1.
    # constrained beam-search decoding by calling constrained_beam_search(), if constraints!=None or force_words_ids!=None.
    gen_kwargs = {
        "max_length": args.max_target_length,
        "num_beams": args.num_beams,
    }
    preds = []
    refs = raw_datasets["validation"]["title"]
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                **gen_kwargs,
            )
            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            generated_tokens = accelerator.gather(generated_tokens)
            generated_tokens = generated_tokens.cpu().numpy()
            
            # decode to sentences
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            preds += decoded_preds
    
    rouge_score = calculate_rouge(preds, refs, avg=True)
    eval_rouge_history["rouge-1"].append(rouge_score['rouge-1']['f'])
    eval_rouge_history["rouge-2"].append(rouge_score['rouge-2']['f'])
    eval_rouge_history["rouge-l"].append(rouge_score['rouge-l']['f'])
    eval_rouge = rouge_score['rouge-1']['f'] + rouge_score['rouge-2']['f'] + rouge_score['rouge-l']['f']
    print(f"rouge-1: {rouge_score['rouge-1']}")
    print(f"rouge-2: {rouge_score['rouge-2']}")
    print(f"rouge-l: {rouge_score['rouge-l']}")
    
    # store the best model
    if (eval_rouge > best_eval_rouge):
        best_eval_rouge = eval_rouge
        if args.output_dir is not None:
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)
            if accelerator.is_main_process:
                tokenizer.save_pretrained(args.output_dir)
        

04/30/2022 18:22:06 - INFO - __main__ - ***** Running training *****
04/30/2022 18:22:06 - INFO - __main__ -   Num examples = 21710
04/30/2022 18:22:06 - INFO - __main__ -   Num Epochs = 1
04/30/2022 18:22:06 - INFO - __main__ -   Instantaneous batch size per device = 4
04/30/2022 18:22:06 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 16
04/30/2022 18:22:06 - INFO - __main__ -   Gradient Accumulation steps = 4
04/30/2022 18:22:06 - INFO - __main__ -   Total optimization steps = 1357
1358it [05:33,  4.35it/s]                          Configuration saved in ./mt5_small_model/config.json


rouge-1: {'r': 0.12268706608332361, 'p': 0.32272470717584084, 'f': 0.16191912815909904}
rouge-2: {'r': 0.04514303060386007, 'p': 0.12627125646634657, 'f': 0.059596704048140305}
rouge-l: {'r': 0.11579522019651808, 'p': 0.3094296574220564, 'f': 0.15352898939504617}


Model weights saved in ./mt5_small_model/pytorch_model.bin
tokenizer config file saved in ./mt5_small_model/tokenizer_config.json
Special tokens file saved in ./mt5_small_model/special_tokens_map.json
Copy vocab file to ./mt5_small_model/spiece.model


## Save eval rouge history

In [13]:
with open(os.path.join(args.output_dir, "eval_rouge.json"), "w") as f:
    json.dump(eval_rouge_history, f)
